In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import os, json

def get_google_credentials(scopes):
    """
    Load Google credentials depending on environment.
    - Lokal: baca dari credentials.json
    - GitHub Actions: baca dari secrets (env var)
    """
    if os.path.exists("credentials.json"):
        # Lokal
        creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scopes=scopes)
    else:
        # GitHub Actions (pakai secrets)
        service_account_info = json.loads(os.environ.get["GOOGLE_CREDENTIALS"])
        creds = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scopes=scopes)
    
    return creds

# ====================================
# 1. Setup koneksi Google Sheets
# ====================================
SPREADSHEET_ID = "1xMNjbpQJhh8jTOaNlxPWy9B2nTEMBAURR9Ys3O90jlM"  # GANTI SESUAI ID
WORKSHEET_NAME = "Jadwal Pasdior"  # ganti sesuai worksheet

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = get_google_credentials(scope)
client = gspread.authorize(creds)

# ====================================
# 2. Ambil semua data lalu potong mulai B5
# ====================================
sheet = client.open_by_key(SPREADSHEET_ID).worksheet(WORKSHEET_NAME)
all_data = sheet.get_all_values()  # semua baris dan kolom yang ada isinya

# Potong: mulai baris ke-5 (index 4) dan kolom B–K (index 1 sampai 10)
data = [row[1:11] for row in all_data[4:] if len(row) >= 11]
df = pd.DataFrame(data, columns=["B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]).copy()

# ====================================
# 3. Update kolom F & G berdasarkan J & K
# ====================================
mask_j = df["J"].astype(str).str.strip() != ""
df.loc[mask_j, "F"] = df.loc[mask_j, "J"]

mask_k = df["K"].astype(str).str.strip() != ""
df.loc[mask_k, "G"] = df.loc[mask_k, "K"]

# ====================================
# 4. Data tambahan (O–R)
# ====================================
data_extra = [row[14:18] for row in all_data[4:] if len(row) >= 18]
df_extra = pd.DataFrame(data_extra, columns=["O", "P", "Q", "R"]).copy()

# Sesuaikan ke format utama
df_extra["B"] = df_extra["O"]
df_extra["C"] = df_extra["P"]
df_extra["F"] = df_extra["Q"]
df_extra["G"] = df_extra["R"]
df_extra["D"] = ""
df_extra["E"] = ""
df_extra = df_extra[["B", "C", "D", "E", "F", "G"]].copy()

# Gabungkan
df_all = pd.concat([df[["B", "C", "D", "E", "F", "G"]], df_extra], ignore_index=True)

# ====================================
# 5. Filter kolom G == "Vincent" & tanggal >= hari ini
# ====================================
df_vincent = df_all[df_all["G"] == "Vincent"].copy()

month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Sept": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

# Bersihkan & parse tanggal
b_str = (
    df_vincent["B"]
    .astype(str)
    .str.strip()
)
b_str_num = b_str.replace(month_map, regex=True)
b_dt = pd.to_datetime(b_str_num, dayfirst=True, errors="coerce")

# Fallback jika serial number Excel
serial_mask = b_str.str.match(r"^\d{4,6}$", na=False)
b_dt.loc[serial_mask] = (
    pd.to_datetime("1899-12-30") +
    pd.to_timedelta(b_str.loc[serial_mask].astype(int), unit="D")
)

df_vincent.loc[:, "B_dt"] = b_dt

# Filter tanggal >= hari ini
today_jkt = datetime.now().date()
df_vincent = df_vincent[df_vincent["B_dt"].dt.date >= today_jkt].copy()

# Sorting by tanggal
df_vincent = df_vincent.sort_values(by="B_dt").reset_index(drop=True)

# ====================================
# 6. Format Output
# ====================================
df_clean = df_vincent[["B", "C", "D", "E", "F", "G", "B_dt"]].copy()
df_clean.columns = ["Tanggal", "Jam", "Anamnesis", "Cara Tobat", "Koor", "Organis", "tgl-format"]

# Nama hari (Indonesia)
import locale
try:
    locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
except:
    try:
        locale.setlocale(locale.LC_TIME, "id_ID")
    except:
        pass

df_clean["Hari"] = df_clean["tgl-format"].dt.strftime("%A")

# Reorder kolom
df_clean = df_clean[["Hari", "Tanggal", "Jam", "Anamnesis", "Cara Tobat", "Koor", "Organis"]]

# Tambahkan kolom Weekday (yes/no)
df_clean["Weekday"] = df_clean["Hari"].apply(lambda x: "yes" if x not in ["Sabtu", "Minggu", "Saturday", "Sunday"] else "no")

# ====================================
# 7. Output
# ====================================
print("📊 Data terformat:")
display(df_clean)

# ====================================
# 8. Simpan hasil ke Google Sheet lain
# ====================================
SPREADSHEET_ID_OUTPUT = "1nqY5jNzJvsy7v37jnb-rlSDUNvsLYiuHq5-ryAW1Kxs"  # Ganti ID sheet tujuan
WORKSHEET_OUTPUT = "jadwal"  # Ganti nama sheet tujuan

# Pastikan worksheet tujuan ada, kalau tidak buat baru
try:
    sheet_out = client.open_by_key(SPREADSHEET_ID_OUTPUT).worksheet(WORKSHEET_OUTPUT)
except gspread.exceptions.WorksheetNotFound:
    sheet_out = client.open_by_key(SPREADSHEET_ID_OUTPUT).add_worksheet(
        title=WORKSHEET_OUTPUT,
        rows=str(len(df_clean) + 10),
        cols=str(len(df_clean.columns) + 5)
    )

# Hapus isi lama, lalu tulis data baru
sheet_out.clear()
sheet_out.update(
    [df_clean.columns.tolist()] + df_clean.astype(str).values.tolist()
)

# Tambahkan Last Update di satu cell (misalnya di J1, supaya tidak ganggu tabel)
from datetime import datetime
from pytz import timezone

tz = timezone("Asia/Jakarta")
last_update_str = f"Last Update: {datetime.now(tz).strftime('%d-%b-%Y %H:%M:%S WIB')}"
sheet_out.update_acell('J1', last_update_str)

print(f"✅ Data berhasil disimpan ke Google Sheet ID: {SPREADSHEET_ID_OUTPUT}, Sheet: {WORKSHEET_OUTPUT}")




📊 Data terformat:


,Hari,Tanggal,Jam,Anamnesis,Cara Tobat,Koor,Organis,Weekday
0,Minggu,24-Aug-2025,10.00,2B,3,Faustina (6),Vincent,no
1,Minggu,31-Aug-2025,07.30,3A,2,Theresia (1),Vincent,no
2,Sabtu,13-Sep-2025,17.00,1,1,Theresia (1),Vincent,no
3,Rabu,24-Sep-2025,18.00,,,Albertus (3),Vincent,yes
4,Jumat,26-Sep-2025,18.00,,,John Choir (5),Vincent,yes
5,Minggu,5-Oct-2025,07.30,3A,4,Theresia (1),Vincent,no
6,Minggu,19-Oct-2025,17.00,1,2,Albertus (3),Vincent,no
7,Minggu,2-Nov-2025,10.00,2B,4,Faustina (6),Vincent,no
8,Minggu,9-Nov-2025,17.00,3A,1,Koor St. Agatha,Vincent,no
9,Sabtu,29-Nov-2025,17.00,2A,1,Faustina (6),Vincent,no


C:\Users\vince\AppData\Local\Temp\ipykernel_9308\1396313597.py:157: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet_out.update(


✅ Data berhasil disimpan ke Google Sheet ID: 1nqY5jNzJvsy7v37jnb-rlSDUNvsLYiuHq5-ryAW1Kxs, Sheet: jadwal
